In [1]:
# Install required packages (if not installed)
#!pip install chromadb pandas sentence-transformers transformers

# Imports
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# === Load CSV Data ===
filepath = '/content/pace_graduate_programs (1).csv'  # <- Make sure you upload the file
df = pd.read_csv(filepath)

program_names = df['Program Name'].tolist()
program_links = df['Program Link'].tolist()

# === Embed Program Names ===
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(program_names)

# === Initialize Chroma Vector Store ===
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.create_collection(name="pace_programs")

# Add documents with metadata (links)
collection.add(
    documents=program_names,
    metadatas=[{"link": link} for link in program_links],
    embeddings=embeddings.tolist(),
    ids=[f"id{i}" for i in range(len(program_names))]
)

# === Semantic Search Function ===
def search_programs(user_query, embedding_model, top_k=5):
    query_embed = embedding_model.encode([user_query])[0].tolist()
    results = collection.query(
        query_embeddings=[query_embed],
        n_results=top_k
    )
    programs = results['documents'][0]
    metadata = results['metadatas'][0]
    return list(zip(programs, [meta['link'] for meta in metadata]))

# === Load RAG Model ===
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
rag_pipeline = pipeline("text2text-generation", model=llm_model, tokenizer=tokenizer)

# === RAG Answer Generator ===
def generate_answer_with_flan(query, program_tuples):
    context = "\n".join([f"{name} – {link}" for name, link in program_tuples])
    prompt = f"""You are a helpful assistant that answers questions about graduate programs at Pace University.

Programs:
{context}

Question: {query}

Answer:"""
    result = rag_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    return result

# === Greeting and Exit Helpers ===
def is_exit(text):
    return text.strip().lower() in ['exit', 'quit', 'bye', 'see you later']

def is_greeting(text):
    return any(greet in text.lower() for greet in ['hi', 'hello', 'hey'])
import json
import uuid

import requests

def save_application(data):
    url = "https://9xsmb17nig.execute-api.us-east-1.amazonaws.com/chatbot"  # Replace with your real endpoint
    response = requests.post(url, json=data)

    if response.status_code == 200:
        return json.loads(response.text)['application_id']
    else:
        print("❌ Error saving application:", response.text)
        return "ERROR"


# === Application Flow ===
def collect_application():
    print("\n🎓 Let's start your application to Pace University!")
    full_name = input("👉 Full Name: ")
    email = input("📧 Email Address: ")
    education = input("🎓 Previous Education (e.g., BSc Computer Science): ")

    # Search for program selection
    while True:
        program_query = input("🧭 What program are you applying to? (type a keyword): ")
        matches = search_programs(program_query, model)
        if not matches:
            print("❌ No matching programs found. Try again.")
            continue
        print("\n✅ Top Matches:")
        for i, (name, link) in enumerate(matches):
            print(f"  {i+1}. {name} – {link}")
        choice = input("👉 Enter the number of the program to apply to: ")
        try:
            program_choice = matches[int(choice) - 1][0]
            break
        except:
            print("⚠️ Invalid choice. Try again.")

    application = {
        "full_name": full_name,
        "email": email,
        "education": education,
        "program": program_choice
    }

    app_id = save_application(application)
    print(f"\n📨 Your application has been saved with ID: {app_id}")
    print("✅ Thank you for applying to Pace University!\n")

# === Updated Chatbot Interface ===
print("🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.")
print("🎓 Ask me anything like 'Do you offer data science?' or type 'apply' to submit an application. Type 'exit' to leave.\n")

while True:
    user_input = input("You: ")

    if is_exit(user_input):
        print("PaceBot: Chat with you later! 🎓")
        break
    elif is_greeting(user_input):
        print("PaceBot: Hello! How can I help you today?")
    elif user_input.strip().lower() == "apply":
        collect_application()
    else:
        program_matches = search_programs(user_input, model)
        response = generate_answer_with_flan(user_input, program_matches)
        print("PaceBot:", response)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.0 MB/s eta

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.
🎓 Ask me anything like 'Do you offer data science?' or type 'apply' to submit an application. Type 'exit' to leave.

You: exit
PaceBot: Chat with you later! 🎓


In [18]:
# === RAG Answer Generator ===
def generate_answer_with_flan(query, program_tuples):
    context = "\n".join([f"{name} – {link}" for name, link in program_tuples])
    prompt = f"""You are a helpful assistant that answers questions about graduate programs at Pace University.

Programs:
{context}

Question: {query}

Answer:"""
    result = rag_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    return result

# === Greeting and Exit Helpers ===
def is_exit(text):
    return text.strip().lower() in ['exit', 'quit', 'bye', 'see you later']

def is_greeting(text):
    return any(greet in text.lower() for greet in ['hi', 'hello', 'hey'])
import json
import uuid

import requests

def save_application(data):
    url = "https://9xsmb17nig.execute-api.us-east-1.amazonaws.com/chatbot"  # Replace with your real endpoint
    response = requests.post(url, json=data)

    if response.status_code == 200:
        try:
            # Attempt to parse JSON
            response_json = json.loads(response.text)
            # Check if the parsed response is a dictionary and contains the key
            if isinstance(response_json, dict) and 'application_id' in response_json:
                return response_json['application_id']
            else:
                print("❌ Unexpected response format or missing 'application_id' key.")
                print("Response text:", response.text) # Print response text for debugging
                return "ERROR"
        except json.JSONDecodeError as e:
            # Handle cases where response is not valid JSON
            print(f"❌ Error parsing response as JSON: {e}")
            print("Response text:", response.text) # Print response text for debugging
            return "ERROR"
    else:
        print(f"❌ Error saving application. Status Code: {response.status_code}")
        print("Response text:", response.text) # Print response text for debugging
        return "ERROR"


# === Application Flow ===
def collect_application():
    print("\n🎓 Let's start your application to Pace University!")
    full_name = input("👉 Full Name: ")
    email = input("📧 Email Address: ")
    education = input("🎓 Previous Education (e.g., BSc Computer Science): ")

    # Search for program selection
    while True:
        program_query = input("🧭 What program are you applying to? (type a keyword): ")
        matches = search_programs(program_query, model)
        if not matches:
            print("❌ No matching programs found. Try again.")
            continue
        print("\n✅ Top Matches:")
        for i, (name, link) in enumerate(matches):
            print(f"  {i+1}. {name} – {link}")
        choice = input("👉 Enter the number of the program to apply to: ")
        try:
            program_choice = matches[int(choice) - 1][0]
            break
        except (ValueError, IndexError):
            print("⚠️ Invalid choice. Try again.")

    application = {
        "full_name": full_name,
        "email": email,
        "education": education,
        "program": program_choice
    }

    app_id = save_application(application)
    print(f"\n📨 Your application has been saved with ID: {app_id}")
    print("✅ Thank you for applying to Pace University!\n")

# === Updated Chatbot Interface ===
print("🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.")
print("🎓 Ask me anything like 'Do you offer data science?' or type 'apply' to submit an application. Type 'exit' to leave.\n")

while True:
    user_input = input("You: ")

    if is_exit(user_input):
        print("PaceBot: Chat with you later! 🎓")
        break
    elif is_greeting(user_input):
        print("PaceBot: Hello! How can I help you today?")
    elif user_input.strip().lower() == "apply":
        collect_application()
    else:
        program_matches = search_programs(user_input, model)
        response = generate_answer_with_flan(user_input, program_matches)
        print("PaceBot:", response)

🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.
🎓 Ask me anything like 'Do you offer data science?' or type 'apply' to submit an application. Type 'exit' to leave.

You: apply

🎓 Let's start your application to Pace University!
👉 Full Name: happymore
📧 Email Address: chytu@gmail.com
🎓 Previous Education (e.g., BSc Computer Science): maths
🧭 What program are you applying to? (type a keyword): accounting

✅ Top Matches:
  1. Accounting Data Analytics and Technology, MS – https://www.pace.edu/program/accounting-data-analytics-and-technologies-ms
  2. Accounting, Public Accounting, CPA Prep, MS – https://www.pace.eduhttps://online.pace.edu/graduate-programs/ms-in-accounting/
  3. Accounting, Public Accounting, CPA Prep, MS – https://www.pace.edu/program/accounting-public-accounting-cpa-prep-ms
  4. Financial Management, MS – https://www.pace.edu/program/financial-management-ms
  5. Accounting, Public Accounting, CPA Prep, MBA – https://www.pace.edu/p